In [ ]:
import os
import json
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
import torch.nn as nn
import torch
from pycocotools import mask
import skimage.measure as measure
from scipy import stats
from tqdm import tqdm

In [ ]:
# output.csv 읽기
submission1 = pd.read_csv('./output1.csv', index_col=None)
submission2 = pd.read_csv('./output2.csv', index_col=None)
submission3 = pd.read_csv('./output3.csv', index_col=None)

subs = [submission1, submission2, submission3]

# 데이터 경로
dataset_path = '/opt/ml/level2_semanticsegmentation_cv-level2-cv-15/data'


# 클래스
classes = ['Backgroud', 'General trash', 'Paper', 'Paper pack', 'Metal',
            'Glass', 'Plastic', 'Styrofoam', 'Plastic bag', 'Battery', 'Clothing']

# 클래스 색깔
palette = [
    [0, 0, 0], [192, 0, 128], [0, 128, 192], [0, 128, 64], [128, 0, 0], [64, 0, 128],
    [64, 0, 192], [192, 128, 64], [192, 192, 128], [64, 64, 128], [128, 0, 192]
]


In [ ]:
# Ensemble 코드

print("Start Ensemble...")

# sample_submisson.csv 열기
submission = pd.read_csv('./code/submission/sample_submission.csv', index_col=None)


for ind in tqdm(range(819)):
    dmasks = []
    
    pseudo_mask = np.zeros((256, 256), dtype=np.int32)
    
    # (256, 256) 를 N개 쌓아서 (N, 256, 256) 로 만든후 최빈값 뽑아내기
    for i in range(len(subs)):
        dmask = np.fromstring(subs[i]['PredictionString'][ind], dtype=int, sep=' ').reshape(256, -1)
        dmasks.append(dmask)
    
    dmasks = np.stack(dmasks)
    m = stats.mode(dmasks)
    pseudo_mask = m[0].squeeze().ravel()
    
    file_name = subs[i]['image_id'][ind]
    predictString = ' '.join(str(e) for e in pseudo_mask.tolist())
    
    submission = submission.append({"image_id" : file_name, "PredictionString" : predictString}, ignore_index=True)
    
# submission.csv로 저장
submission.to_csv("./code/submission/Ensemble.csv", index=False)


print("Done")